# Manufacturer / Aircraft Type Heatmap

This notebook produces a heatmap of manufacturers by aicraft models.

Individual models within a family are grouped:

- e.g. Both A320-214 and A320-232 will appear under the "A320" family
- Aircraft with fewer sightings than the configured threshold are assigned to the "Other" family

To use the notebook, set the threshold value and export format for charts in the first code cell before running the notebook.

In [ ]:
# Sightings threshold below which aircraft are assigned to a generic "Other" group
prefix_count_threshold = 100

# Export format for the heatmap:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
export_format = "PNG"

In [ ]:
from pathlib import Path
import sqlparse

# Read the query file
query_file_path = Path("sql") / "sightings.sql"
with open(query_file_path.absolute(), "r") as f:
    query = f.read().replace("\n", " ")

# Show a pretty-printed form of the query
print(sqlparse.format(query, reindent=True, keyword_case="upper"))

In [ ]:
import pandas as pd
import sqlite3
import os

# Connect to the database, execute the query and read the results into a dataframe
database_path = os.environ["FLIGHT_RECORDER_DB"]
connection = sqlite3.connect(database_path)
df = pd.read_sql_query(query, connection, parse_dates=["Date"])

# Check there is some data
if not df.shape[0]:
    message = f"No data found"
    raise ValueError(message)

# Preview the data
df.head()

In [ ]:
import re

def extract_prefix(model):
    # Try extracting leading letters and digits (e.g., "A320", "B737")
    match = re.match(r"^([A-Z]+[0-9]+)", str(model).replace(" ", "").upper())
    prefix = match.group(1) if match else None

    # If this doesn"t result in a prefix, try just splitting on the first "-"
    if not prefix:
        tokens = model.split("-", 1)
        prefix = tokens[0] if len(tokens) > 1 else None

    # On its own, the above can result in some nonsensical prefixes so check we do have some numbers in
    # to indicate the model
    if prefix and not any(char.isdigit() for char in prefix):
        prefix = None

    return prefix

# Extract an aircraft type prefix for each sighting
df["Type"] = df["Model"].apply(extract_prefix)

# Preview the data
df.head()

In [ ]:
# Identify counts by prefix and use them to identify common prefixes
prefix_counts = df["Type"].value_counts()
common_prefixes = prefix_counts[prefix_counts >= prefix_count_threshold].index

# Replace uncommon prefixes with "Other"
df["Type"] = df.apply(
    lambda row: row["Type"] if row["Type"] in common_prefixes else "Other",
    axis=1
)

# Preview the data
df.head()

In [ ]:
# Generate the pivot table to act as the source data
pivot_table = df.pivot_table(
    index="Manufacturer",
    columns="Type",
    values="Id",
    aggfunc="count",
    fill_value=0
)

In [ ]:
import pandas as pd
from pathlib import Path

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Create a Pandas Excel writer
export_file_name = "Manufacturer-Model-Heatmap"
output_path = export_folder_path / f"{export_file_name}.xlsx"

with pd.ExcelWriter(output_path.absolute(), engine="openpyxl") as writer:
    # Monthly data
    pivot_table.to_excel(writer, sheet_name="Pivot Table", index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the heatmap
plt.figure(figsize=(12, pivot_table.shape[0] / 3))
sns.heatmap(pivot_table, annot=False, fmt="d", cmap="YlOrRd")
plt.title("Aircraft Sightings by Manufacturer and Group (Auto-Generated)")
plt.ylabel("Manufacturer")
plt.xlabel("Aircraft Group")
plt.xticks(rotation=90)
plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()